In [ ]:
%load_ext autoreload
%autoreload

In [2]:
from modeling import process_dataset, produce_idefics_dataset

train_data_path = '/mnt/u14157_ic_nlp_001_files_nfs/nlpdata1/home/ismayilz/cs503-project/data/train/nuscenes/v1_1_train_nus_ext_converted.json'

train_dataset = process_dataset(train_data_path, apply_context="chain")
train_idefics_dataset = produce_idefics_dataset(train_dataset)

100%|██████████| 696/696 [00:00<00:00, 2178.74it/s]


In [3]:
import json
from transformers import AutoProcessor

model_dir = "HuggingFaceM4/idefics2-8b"

processor = AutoProcessor.from_pretrained(
    model_dir,
    do_image_splitting=False
)

# with open("./data/nuscenes/train_idefics_redcircle_vb_chain.json") as f:
#     data = json.load(f)

from modeling import GVQADataCollator

c = GVQADataCollator(processor)
c(train_idefics_dataset[:10])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prompt:
Context: None
Task:
What are the important objects in the current scene? Those objects will be considered for the future reasoning and driving decision.
text:
User:CAM_BACK: <image> CAM_BACK_LEFT: <image> CAM_BACK_RIGHT: <image> CAM_FRONT: <image> CAM_FRONT_LEFT: <image> CAM_FRONT_RIGHT: <image> Context: None
Task:
What are the important objects in the current scene? Those objects will be considered for the future reasoning and driving decision.<end_of_utterance>
Assistant: There is a brown SUV to the back of the ego vehicle, a black sedan to the back of the ego vehicle, and a green light to the front of the ego vehicle. The IDs of these objects are <c1,CAM_BACK,1088.3,497.5>, <c2,CAM_BACK,864.2,468.3>, and <c3,CAM_FRONT,1043.2,82.2>.<end_of_utterance>

Prompt:
Context: None
Task:
What is the moving status of the object marked with red circle? Please select the correct answer from the following options: A. Stopped. B. Reverse parking. C. Turn left. D. Back up.
text:
User:CAM_BA

{'input_ids': tensor([[    0,     0,     0,  ..., 28750, 13902, 32002],
        [    0,     0,     0,  ..., 28747,   334, 32002],
        [    1,  1247, 28747,  ...,  4975, 28723, 32002],
        ...,
        [    0,     0,     0,  ..., 28747,   365, 32002],
        [    0,     0,     0,  ..., 28783, 13902, 32002],
        [    0,     0,     0,  ...,  4975, 28723, 32002]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'pixel_values': tensor([[[[[ 0.0588,  0.0431,  0.0510,  ..., -0.0667, -0.0588, -0.0510],
           [ 0.0510,  0.0275,  0.0824,  ..., -0.0745, -0.0588, -0.0431],
           [ 0.0353,  0.0353,  0.0588,  ..., -0.0588, -0.0588, -0.0588],
           ...,
           [-0.5216, -0.5137, -0.5294,  ..., -0.6078, -0.6078, -0.6078],
           [-0.5294, -0.5216, -0.5294,  ..., -0.5922, -0.5922,